In [48]:
import numpy as np
import os
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from imblearn.over_sampling import RandomOverSampler


In [49]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))    
    x_test = np.load(os.path.join(folder, 'x_test.npy'))    
    y_test = np.load(os.path.join(folder, 'y_test.npy'))    
    return x_train, y_train, x_test, y_test

In [81]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogisticRegression:
    def __init__(self, dim=2):
        rng = np.random.default_rng(seed=0)
        self.w = rng.normal(size=(dim, 1)) / np.sqrt(dim)
        self.b = np.zeros((1,))
        
    def predict(self, x, probs=False):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"
        
        x = x.dot(self.w) + self.b  # logits
        p = sigmoid(x)  # probabilities
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')
        
    def fit(self, x, y, iters=1000, lr=0.01):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        # y - np.array размернсоти [N]
        #     Массив меток (правильных ответов).
        assert len(x) == len(y), \
            "Количество экземпляров в массиве X не равно количеству меток в массиве Y. " + \
            f"Полученные размеры: len(X) = {len(x)}, len(Y) = {len(y)}."
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"
        # Алгоритм градиентного спуска.
        # Минимизируется бинарная кросс-энтропия.
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)
        return self

## 1. Применение логистической регрессии (несбалансированные данные)

### 1.1 Создание и обучение логистической регрессии

In [51]:
# Указание: производить нормализацию данных не нужно, это часть задания.
x_train, y_train, x_test, y_test = load_data('dataset1')

In [52]:
# Создайте модель логистической регрессии и обучите её, используя метод fit.
logisticRegression = LogisticRegression(x_train.shape[1])# создаем модель, передавая кол-во признаков у объекта
logisticRegression.fit(x_train, y_train)# обучаем на тренировочных данных

In [53]:
# Получите предсказания на тестовой выборке и оцените точность модели, 
# используя accuracy_score из пакета SciKit-Learn.
Y = logisticRegression.predict(x_test)
accuracy = accuracy_score(y_test, Y)
accuracy

0.9045454545454545

### 1.2 Анализ качества модели

In [54]:
# Допишите класс "глупого классификатора", что всегда предсказывает класс `0`. 

class DummyClassifier:
    def __init__(self):
        print('Hello, brother!')
        
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Должен возвращаться массив N предсказаний
        return np.zeros((x.shape[0],))
    def fit(self, x, y, iters=1000, lr=0.01):
        pass

In [55]:
# Оцените точность "глупого классификатора", объясните результат.
dummyClassifier = DummyClassifier()
dummyClassifier.fit(x_train, y_train)
Y_Dummy = dummyClassifier.predict(x_test)
accuracy_Dummy = accuracy_score(y_test, Y_Dummy) 
accuracy_Dummy

Hello, brother!


0.9090909090909091

In [56]:
# Используйте дополнительные метрики (f1-score, recall, precision) из пакета sklearn для анализа "глупого классификатора".
f1_score_Dummy = f1_score(y_test, Y_Dummy) 
recall_Dummy = recall_score(y_test, Y_Dummy) 
precision_score_Dummy = precision_score(y_test, Y_Dummy) 
print(f"f1_score: {f1_score_Dummy}\n")
print(f"recall_score: {recall_Dummy}\n")
print(f"precision_score: {precision_score_Dummy}\n")

f1_score: 0.0

recall_score: 0.0

precision_score: 0.0



c:\Users\Павел\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [57]:
# Используя те же метрики, проанализируйте обученную вами модель логистической регрессии.
 
print(f"f1_score: {f1_score(y_test, Y)}\n")
print(f"recall_score: {recall_score(y_test, Y) }\n")
print(f"precision_score: {precision_score(y_test, Y)}\n")

f1_score: 0.4

recall_score: 0.35

precision_score: 0.4666666666666667



In [41]:
# Объясните результат, описав его комментариями в этой клетке.

# В наборе данных присутсвует дисбаланс классов, из-за которыго accuracy вернул не объективную оценку
# Это показывает, что для получения более объективной оценки работы классификатора, необходимо использовать несколько метрик

### 1.3 Анализ набора данных

In [58]:
# Посчитайте количество экземпляров данных для каждого класса.
print(f"Count of class 1: {sum(y_test)}\nCount of class 2: {y_test.shape[0] - sum(y_test)} ")

Count of class 1: 20.0
Count of class 2: 200.0 


In [59]:
# Предложите способ улучшения качества модели. Подсказка: добавление дубликатов в данные.
# Указание: не изменяйте тестовую выборку.

oversample = RandomOverSampler(sampling_strategy='minority')
x_resampled, y_resampled = oversample.fit_resample(x_train, y_train)
print(f"Count of class 1 resampled: {sum(y_resampled)}\nCount of class 2 resampled: {y_resampled.shape[0] - sum(y_resampled)} ")

Count of class 1 resampled: 200.0
Count of class 2 resampled: 200.0 


In [65]:
# Создайте и обучите модель с использованием предложенных наработок.
logisticRegressionForResampledData = LogisticRegression(x_resampled.shape[1])
logisticRegressionForResampledData.fit(x_resampled, y_resampled)

In [66]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics. 
# Указание: постарайтесь сбалансировать данные таким образом, чтобы новая модель была ощутимо лучше старой.
Y_resampled = logisticRegressionForResampledData.predict(x_test)

print(f"accuracy_score: {accuracy_score(y_test, Y_resampled)}\n")
print(f"f1_score: {f1_score(y_test, Y_resampled)}\n")
print(f"recall_score: {recall_score(y_test, Y_resampled)}\n")
print(f"precision_score: {precision_score(y_test, Y_resampled)}\n")

accuracy_score: 0.95

f1_score: 0.7317073170731707

recall_score: 0.75

precision_score: 0.7142857142857143



## 2. Применение логистической регрессии (нелинейные данные)

In [67]:
x_train, y_train, x_test, y_test = load_data('dataset2')

In [68]:
# Создайте и обучите модель но этом наборе данных.
logisticRegression2 = LogisticRegression(x_train.shape[1])
logisticRegression2.fit(x_train, y_train)
Y2 = logisticRegression2.predict(x_test)

In [69]:
# Проанализируйте качество модели.
print(f"accuracy_score: {accuracy_score(y_test, Y2)}\n")
print(f"f1_score: {f1_score(y_test, Y2)}\n")
print(f"recall_score: {recall_score(y_test, Y2)}\n")
print(f"precision_score: {precision_score(y_test, Y2)}\n")

accuracy_score: 0.57

f1_score: 0.6194690265486725

recall_score: 0.7777777777777778

precision_score: 0.5147058823529411



In [70]:
# FEATURE ENGINEERING: попробуйте применить на исходных данных разные нелинейные функции (sin, tanh, ...).
# Объедините трансформированные данные с исходными (важно: количество экземпляров в x_train не должно увеличиться).
sign_x_train = x_train[:, 0].reshape(x_train.shape[0], 1)
sign_y_train = np.cos(x_train[:, 1]).reshape(x_train.shape[0], 1)# применяю на втором признаке объектов из тренировочной выборки косинус

sign_x_test = x_test[:, 0].reshape(x_test.shape[0], 1)
sign_y_test = np.cos(x_test[:, 1]).reshape(x_test.shape[0], 1)# применяю на втором признаке объектов из тестовой выборки косинус

x_train_transformed = np.hstack((sign_x_train, sign_y_train))
x_test_transformed = np.hstack((sign_x_test, sign_y_test))


In [71]:
# Создайте и обучите модель с использованием наработок.
sign_logisticRegression = LogisticRegression(x_train.shape[1])
sign_logisticRegression.fit(x_train_transformed, y_train)
sign_Y = sign_logisticRegression.predict(x_test_transformed)

In [72]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics. 
# Указание: постарайтесь добиться точности в 100%!
print(f"accuracy_score: {accuracy_score(y_test, sign_Y)}\n")
print(f"f1_score: {f1_score(y_test, sign_Y)}\n")
print(f"recall_score: {recall_score(y_test, sign_Y)}\n")
print(f"precision_score: {precision_score(y_test, sign_Y)}\n")

accuracy_score: 1.0

f1_score: 1.0

recall_score: 1.0

precision_score: 1.0



## 3. Доп. задания (любое на выбор, опционально)

### 3.1 'Упрощение' логистической регрессии

Сложность: легко.

In [73]:
"""
Модифицируйте класс логистической регрессии так, чтобы в нём не использовалась сигмоида.
То есть вывод о предсказанном классе должен делаться на основе значений "до сигмоиды".
Вспомогательная ссылка: https://en.wikipedia.org/wiki/Logit
"""

class LogisticRegression:
    def __init__(self, dim=2):
        self.w = np.random.randn(dim, 1) / np.sqrt(dim)
        self.b = np.zeros((1,))
        
    def predict(self, x, probs=False):
        x = x.dot(self.w) + self.b
        return np.array(x > 0.0).astype('int32')# если x больше 0, то будет 1
        
    def fit(self, x, y, iters=100, lr=0.01):
        y = y.reshape(-1, 1)
        for _ in range(iters):
            preds = self.predict(x)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)

In [74]:
# Указание: производить нормализацию данных не нужно, это часть задания.
x_train, y_train, x_test, y_test = load_data('dataset1')

In [75]:
# Перенесите обученные веса модели из пункта 1.3 в новую модель с модифицированным кодом
ModifyLogisticRegression = LogisticRegression(x_resampled.shape[1])
ModifyLogisticRegression.fit(x_resampled, y_resampled)
Y_modify = ModifyLogisticRegression.predict(x_test)

In [76]:
# Убедитесь, что предсказания модели с модифицированными кодом совпадают с предсказаниями
# модели из пункта 1.3
print(f"Result of MODIFIED Logistic Regression:")
print(f"  accuracy_score: {accuracy_score(y_test, Y_modify)}")
print(f"  f1_score: {f1_score(y_test, Y_modify)}")
print(f"  recall_score: {recall_score(y_test, Y_modify)}")
print(f"  precision_score: {precision_score(y_test, Y_modify)}")

print(f"\nResult of DEFAULT Logistic Regression:")
print(f"  accuracy_score: {accuracy_score(y_test, Y_resampled)}")
print(f"  f1_score: {f1_score(y_test, Y_resampled)}")
print(f"  recall_score: {recall_score(y_test, Y_resampled)}")
print(f"  precision_score: {precision_score(y_test, Y_resampled)}")

Result of MODIFIED Logistic Regression:
  accuracy_score: 0.9409090909090909
  f1_score: 0.6976744186046512
  recall_score: 0.75
  precision_score: 0.6521739130434783

Result of DEFAULT Logistic Regression:
  accuracy_score: 0.95
  f1_score: 0.7317073170731707
  recall_score: 0.75
  precision_score: 0.7142857142857143


**Как видно из выбранных метрик, модифицированная модель предсказывает приблизительно с той же точностью, что и обычная**  

### 3.2 'Обобщение' логистической регрессии

Напишите многоклассовый классификатор. Обучите его на наборе данных ниже.

In [77]:
x_train, y_train, x_test, y_test = load_data('dataset3')

<b>Ансамбль логистических регрессий.</b> Сложность: супергерой.

In [83]:
"""
Напишите класс, что инкапсулирует в себе `C` логистических регрессий, 
где `C` - количество классов. i-ая логистическая регрессия производит 
бинарную классификацию вида: все остальные классы и i-ый класс.
"""

class MulticlassLogisticRegression:
    def __init__(self, n_classes, dim):
        """
        n_classes: количество классов;
        dim: количество признаков.
        """
        self.n_classes = n_classes
        self.dim = dim
        # Для каждого класса создаем создаем свой экземпляр логистической регрессии 
        self.models = [LogisticRegression(dim) for _ in range(n_classes)]
    
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для 

        # Получаем предсказания вероятностей для каждого класса
        predictions = np.array([model.predict(x, probs=True).flatten() for i, model in enumerate(self.models)]).T
        return np.argmax(predictions, axis=1)
    
    def fit(self, x, y, iters=1000, lr=0.01):
        for i in range(self.n_classes):
            # Помечаем i-ый класс единицей остальные нулями 
            y_bin = (y == i).astype(int)
            self.models[i].fit(x, y_bin, iters=iters, lr=lr)

In [84]:
# Создайте и обучите написанный классификатор.
n_classes = len(np.unique(y_train))
multiclassLogisticRegression = MulticlassLogisticRegression(n_classes, np.shape(x_train)[1])
multiclassLogisticRegression.fit(x_train, y_train)

In [86]:
#  Оцените точность модели.
Y_Multi = multiclassLogisticRegression.predict(x_test)

print(f"\nResult of Multiple Logistic Regression:")
print(f"  accuracy_score: {accuracy_score(y_test, Y_Multi)}")


Result of Multiple Logistic Regression:
  accuracy_score: 0.9333333333333333


<b>Softmax классификатор.</b> Сложность: математический гений.

In [28]:
"""
Напишите класс классификатора, основанного на функции Softmax.
Алгоритм работы данного классификатора:
x - вектор (экземпляр данных) размерности dim.
W - матрица весов размерности [dim, n_classes].

Ответ классификатора формируется как:
logits = x * W - матричное умножение
p = softmax(logits)
class_id = argmax(p)

Для данного классификатора требуется модифицировать алгоритм обучения в методе fit.

Вспомогательные ресурсы:
https://en.wikipedia.org/wiki/Softmax_function
https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
"""

class SoftmaxClassificator:
    def __init__(self, n_classes, dim):
        pass
    
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для 
        # i-го экземпляра данных в `x`.
        pass
    
    def fit(self, x, y):
        pass

In [29]:
# Создайте и обучите написанный классификатор. Оцените точность модели, посчитайте матрицу ошибок (выведите её с помощью matplotlib).


In [30]:
# Создайте и обучите написанный классификатор на наборе данных из задания 1 (опционально). 
# Оцените точность модели, посчитайте матрицу ошибок (выведите её с помощью matplotlib).
